In [1]:
import os
from pydub import AudioSegment
import whisper
from pathlib import Path

c:\Users\Jason\anaconda3\envs\General\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
def chunk_audio(file_path, chunk_length_ms=60000):
    audio = AudioSegment.from_file(file_path, format="mp3")
    length_audio = len(audio)
    chunks = []
    for i in range(0, length_audio, chunk_length_ms):
        chunks.append(audio[i:i+chunk_length_ms])
    return chunks

def format_timestamp(seconds):
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"[{hours:02d}:{minutes:02d}:{seconds:02d}]"

def transcribe_chunk(chunk, model, start_time):
    temp_file = Path("temp_chunk.mp3")
    chunk.export(temp_file, format="mp3")
    result = model.transcribe(temp_file)
    temp_file.unlink()  # Removes the file
    
    transcriptions = []
    for segment in result['segments']:
        timestamp = format_timestamp(start_time + segment['start'])
        transcriptions.append(f"{timestamp} {segment['text']}")
    
    return '\n'.join(transcriptions), start_time + result['duration']

def transcribe_audio(file_path, chunk_length_ms=60000):
    model = whisper.load_model("base")  # or any other model size
    audio_chunks = chunk_audio(file_path, chunk_length_ms)
    
    file_path = Path(file_path)
    output_file = file_path.parent / "transcription_output.txt"
    
    start_time = 0
    with output_file.open('w') as f:
        for i, chunk in enumerate(audio_chunks):
            try:
                print(f"Transcribing chunk {i+1}/{len(audio_chunks)}...")
                transcription, start_time = transcribe_chunk(chunk, model, start_time)
                f.write(transcription + '\n')
            except Exception as e:
                print(f"Error transcribing chunk {i+1}: {e}")

In [5]:
path_to_mp3 = Path(r"I:\\TED.mp3")
transcribe_audio(path_to_mp3)

FileNotFoundError: [WinError 2] The system cannot find the file specified